In [3]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from utils import nc_utils, shape_utils

from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/

import numpy as np
import pandas as pd

/Users/haykgrigoryan/Documents/IIAP/Remote-Sensing-Data-Processing-Services/RemoteSensing


In [3]:
nc_fid = Dataset('data/example.nc', 'r')

nc_attrs, nc_dims, nc_vars = nc_utils.ncdump(nc_fid,True)

lats = nc_fid.variables['lat'][:]
lons = nc_fid.variables['lon'][:]

NetCDF Global Attributes:
	ncei_template_version: 'NCEI_NetCDF_Grid_template_V2.0'
	title: 'NOAA Climate Data Record (CDR) of CPC Morphing Technique (CMORPH) High-Resolution Global Precipitation Estimates'
	keywords: 'Precipitation, Satellite, High-Resolution, Global'
	summary: 'The CMORPH CDR is a reprocessed and bias-corrected global precipitation product created on an 8kmx8km grid over the globe (60S-60N) and in a 30-minute temporal resolution for an 18-year period from January 1998 to the present. First, the purely satellite based CMORPH precipitation estimates (raw CMORPH) are reprocessed. The integration algorithm is fixed and the input Level 2 passive microwave (PMW) retrievals of instantaneous precipitation rates are from identical versions throughout the entire data period. Bias correction is then performed for the raw CMORPH through probability density function (PDF) matching against the CPC daily gauge analysis over land and through adjustment against the Global Precipitatio

In [4]:
shapefile = shape_utils.readShapefile("../../Armenia/arm_admbnda_adm0_2019.shp")
print(shapefile)

# Get the bounding box for each geometry
bounds = shapefile.bounds

# Find the overall min and max latitude and longitude
min_lon = bounds['minx'].min()
max_lon = bounds['maxx'].max()
min_lat = bounds['miny'].min()
max_lat = bounds['maxy'].max()

lat_mask = (lats >= min_lat) & (lats <= max_lat)
lon_mask =(lons >= min_lon) & (lons <= max_lon)
lat_indexes = np.where(lat_mask)[0]
lon_indexes = np.where(lon_mask)[0]
points = pd.DataFrame({})

for i in range(lat_indexes.shape[0]):
    for j in range(lon_indexes.shape[0]):
        lat = lats[lat_indexes[i]]
        lon = lons[lon_indexes[j]]
        if shape_utils.isPointInsideShapefile(shapefile, lat, lon):
            points = pd.concat([points, pd.DataFrame({'lat_index': lat_indexes[i], 'lat': lat, 'lon_index': lon_indexes[j], 'lon': lon}, index=[0])], ignore_index=True)
            
points.to_csv('points.csv', index=False)

   OBJECTID              ADM0_EN                    ADM0_HY  \
0         1  Republic of Armenia  Հայաստանի Հանրապետություն   

              ADM0_RU ADM0_PCODE  Shape_Leng  Shape_Area  \
0  Республика Армения         AM   18.254547    3.015704   

                                            geometry  
0  POLYGON ((44.82808 41.29903, 44.82929 41.29881...  
